# 🏋️ Biceps Curl Form Classifier Training

Bu notebook, videolardan açı çıkararak RandomForest Classifier modeli eğitir.

## Pipeline:
1. **Video İşleme**: MediaPipe ile frame-by-frame açı çıkarımı
2. **Feature Extraction**: CSV'lerden 16 biyomekanik özellik çıkarımı
3. **Model Eğitimi**: RandomForest Classifier ile sınıflandırma
4. **Değerlendirme**: Accuracy, confusion matrix, feature importance

## 📦 1. Paket Kurulumu

In [ ]:
##  2. Google Drive Bağlantısı & Video Yapısı

Videoların dizin yapısı şu şekilde olmalı:
`
/content/drive/MyDrive/normal_video/
 train/
    true/
       true_1.mp4, true_2.mp4, ...
    false/
        false_1.mp4, false_2.mp4, ...
 test/
    true/ & false/
 validation/
     true/ & false/
`

## 📁 2. Google Drive Bağlantısı & Video Yapısı

Videoların dizin yapısı şu şekilde olmalı:
```
/content/drive/MyDrive/videos/
├── true/
│   ├── true_1.mp4
│   ├── true_2.mp4
│   └── ...
└── false/
    ├── false_1.mp4
    ├── false_2.mp4
    └── ...
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Video klasör yollarını ayarlayın
VIDEOS_BASE_DIR = '/content/drive/MyDrive/videos'  # Değiştirin!
TRUE_VIDEOS_DIR = f'{VIDEOS_BASE_DIR}/true'
FALSE_VIDEOS_DIR = f'{VIDEOS_BASE_DIR}/false'
OUTPUT_DIR = '/content/output'

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}/csv/true', exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}/csv/false', exist_ok=True)

print(f"✓ True videos: {TRUE_VIDEOS_DIR}")
print(f"✓ False videos: {FALSE_VIDEOS_DIR}")
print(f"✓ Output: {OUTPUT_DIR}")

## 🎯 3. Ana Sınıflar ve Fonksiyonlar

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from typing import Optional, Tuple, Dict, List
from collections import deque


class BicepsCurlCounter:
    """Rep sayımı ve form değerlendirmesi için sınıf"""
    
    def __init__(self):
        self.left_reps = 0
        self.right_reps = 0
        self.left_correct_reps = 0
        self.right_correct_reps = 0
        self.left_state = 'down'
        self.right_state = 'down'
        self.left_cycle_index = 0
        self.right_cycle_index = 0
        
        # Smoothing
        self.left_angle_history = deque(maxlen=5)
        self.right_angle_history = deque(maxlen=5)
        
        # Thresholds
        self.up_threshold = 60
        self.down_threshold = 140
        self.alignment_threshold = 30
        
    def get_smoothed_angle(self, angle, history):
        if angle is not None:
            history.append(angle)
        if len(history) == 0:
            return None
        return sum(history) / len(history)
    
    def update(self, left_angle, right_angle, left_aligned, right_aligned, left_elbow_align, right_elbow_align):
        # Smooth angles
        left_smoothed = self.get_smoothed_angle(left_angle, self.left_angle_history)
        right_smoothed = self.get_smoothed_angle(right_angle, self.right_angle_history)
        
        # Left arm rep counting
        if left_smoothed is not None:
            if self.left_state == 'down' and left_smoothed < self.up_threshold:
                self.left_state = 'up'
            elif self.left_state == 'up' and left_smoothed > self.down_threshold:
                self.left_state = 'down'
                self.left_reps += 1
                self.left_cycle_index += 1
                if left_aligned and left_elbow_align < self.alignment_threshold:
                    self.left_correct_reps += 1
        
        # Right arm rep counting
        if right_smoothed is not None:
            if self.right_state == 'down' and right_smoothed < self.up_threshold:
                self.right_state = 'up'
            elif self.right_state == 'up' and right_smoothed > self.down_threshold:
                self.right_state = 'down'
                self.right_reps += 1
                self.right_cycle_index += 1
                if right_aligned and right_elbow_align < self.alignment_threshold:
                    self.right_correct_reps += 1
        
        return {
            'left_reps': self.left_reps,
            'right_reps': self.right_reps,
            'left_correct_reps': self.left_correct_reps,
            'right_correct_reps': self.right_correct_reps,
            'total_reps': self.left_reps + self.right_reps,
            'left_state': self.left_state,
            'right_state': self.right_state,
            'left_cycle_index': self.left_cycle_index,
            'right_cycle_index': self.right_cycle_index,
            'left_smoothed_angle': left_smoothed,
            'right_smoothed_angle': right_smoothed
        }


class VideoAngleExtractor:
    """MediaPipe kullanarak videodan açı çıkartıcı"""
    
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            smooth_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
    def calculate_angle(self, p1, p2, p3):
        if None in [p1, p2, p3]:
            return None
        v1 = np.array(p1) - np.array(p2)
        v2 = np.array(p3) - np.array(p2)
        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        return np.degrees(np.arccos(cos_angle))
    
    def get_landmark(self, landmarks, idx, w, h):
        if not landmarks:
            return None
        lm = landmarks.landmark[idx]
        if lm.visibility < 0.5:
            return None
        return (lm.x * w, lm.y * h)
    
    def check_alignment(self, shoulder, elbow, hip, torso_height):
        if None in [shoulder, elbow, hip]:
            return True
        elbow_offset = abs(elbow[0] - shoulder[0])
        max_deviation = torso_height * 0.15
        return elbow_offset < max_deviation
    
    def process_video(self, video_path: str, output_csv_path: str) -> pd.DataFrame:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Video açılamadı: {video_path}")
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        counter = BicepsCurlCounter()
        results = []
        frame_idx = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_idx += 1
            h, w = frame.shape[:2]
            
            # MediaPipe işleme
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pose_results = self.pose.process(frame_rgb)
            landmarks = pose_results.pose_landmarks
            
            # Landmark'ları al
            left_shoulder = self.get_landmark(landmarks, 11, w, h)
            left_elbow = self.get_landmark(landmarks, 13, w, h)
            left_wrist = self.get_landmark(landmarks, 15, w, h)
            left_hip = self.get_landmark(landmarks, 23, w, h)
            
            right_shoulder = self.get_landmark(landmarks, 12, w, h)
            right_elbow = self.get_landmark(landmarks, 14, w, h)
            right_wrist = self.get_landmark(landmarks, 16, w, h)
            right_hip = self.get_landmark(landmarks, 24, w, h)
            
            # Kol açılarını hesapla (biceps curl angle)
            left_arm_angle = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Dirsek hizalama açıları (elbow-shoulder-hip)
            left_elbow_align = self.calculate_angle(left_elbow, left_shoulder, left_hip)
            right_elbow_align = self.calculate_angle(right_elbow, right_shoulder, right_hip)
            
            # True torso angles (hip-shoulder-vertical)
            left_true_torso = None
            right_true_torso = None
            if left_shoulder and left_hip:
                vertical_left = (left_shoulder[0], left_shoulder[1] - 100)
                left_true_torso = self.calculate_angle(vertical_left, left_shoulder, left_hip)
            if right_shoulder and right_hip:
                vertical_right = (right_shoulder[0], right_shoulder[1] - 100)
                right_true_torso = self.calculate_angle(vertical_right, right_shoulder, right_hip)
            
            # Alignment check
            left_torso_height = abs(left_hip[1] - left_shoulder[1]) if left_hip and left_shoulder else 100
            right_torso_height = abs(right_hip[1] - right_shoulder[1]) if right_hip and right_shoulder else 100
            left_aligned = self.check_alignment(left_shoulder, left_elbow, left_hip, left_torso_height)
            right_aligned = self.check_alignment(right_shoulder, right_elbow, right_hip, right_torso_height)
            
            # Rep counter update
            status = counter.update(
                left_arm_angle, right_arm_angle,
                left_aligned, right_aligned,
                left_elbow_align or 999, right_elbow_align or 999
            )
            
            # Row oluştur
            time_sec = frame_idx / fps if fps > 0 else 0
            row = {
                'frame': frame_idx,
                'time_s': round(time_sec, 3),
                'left_cycle_index': status['left_cycle_index'],
                'right_cycle_index': status['right_cycle_index'],
                'left_state': status['left_state'],
                'right_state': status['right_state'],
                'left_angle_raw_deg': round(left_arm_angle, 2) if left_arm_angle else '',
                'right_angle_raw_deg': round(right_arm_angle, 2) if right_arm_angle else '',
                'left_angle_smoothed_deg': round(status['left_smoothed_angle'], 2) if status['left_smoothed_angle'] else '',
                'right_angle_smoothed_deg': round(status['right_smoothed_angle'], 2) if status['right_smoothed_angle'] else '',
                'left_elbow_alignment_angle_deg': round(left_elbow_align, 2) if left_elbow_align else '',
                'right_elbow_alignment_angle_deg': round(right_elbow_align, 2) if right_elbow_align else '',
                'left_true_torso_angle_deg': round(left_true_torso, 2) if left_true_torso else '',
                'right_true_torso_angle_deg': round(right_true_torso, 2) if right_true_torso else '',
                'left_aligned': int(left_aligned),
                'right_aligned': int(right_aligned),
                'left_reps': status['left_reps'],
                'right_reps': status['right_reps'],
                'left_correct_reps': status['left_correct_reps'],
                'right_correct_reps': status['right_correct_reps'],
                'total_reps': status['total_reps']
            }
            results.append(row)
            
            # Progress
            if frame_idx % 50 == 0:
                print(f"  {frame_idx}/{frame_count} ({100*frame_idx/frame_count:.0f}%)", end='\r')
        
        cap.release()
        
        df = pd.DataFrame(results)
        df.to_csv(output_csv_path, index=False)
        
        return df


print("✓ Sınıflar yüklendi!")

## 🎬 4. Tüm Videoları İşle ve CSV Çıkart

In [ ]:
def batch_process_videos(input_folder, output_folder, prefix, category):
    """Videolardan CSV çıkart"""
    os.makedirs(output_folder, exist_ok=True)
    
    video_extensions = ['.mp4', '.avi', '.mov', '.MOV']
    video_files = []
    for ext in video_extensions:
        video_files.extend(list(Path(input_folder).glob(f'{prefix}_*{ext}')))
    
    print(f"\n{'='*60}")
    print(f"{category} VIDEOLARI İŞLENİYOR")
    print(f"Bulunan video sayısı: {len(video_files)}")
    print(f"{'='*60}")
    
    extractor = VideoAngleExtractor()
    successful = 0
    
    for idx, video_path in enumerate(sorted(video_files), 1):
        print(f"\n[{idx}/{len(video_files)}] {video_path.name}")
        
        try:
            # Video numarasını al
            import re
            match = re.search(r'_(\d+)', video_path.stem)
            video_num = match.group(1) if match else str(idx)
            
            output_csv = f"{output_folder}/output_{prefix}_{video_num}.csv"
            df = extractor.process_video(str(video_path), output_csv)
            print(f"  ✓ {len(df)} frame işlendi")
            successful += 1
        except Exception as e:
            print(f"  ✗ Hata: {e}")
    
    print(f"\n✓ {successful}/{len(video_files)} video başarıyla işlendi")
    return successful

# TRUE videoları işle
true_count = batch_process_videos(
    TRUE_VIDEOS_DIR, 
    f'{OUTPUT_DIR}/csv/true', 
    'true', 
    'DOĞRU FORM (true)'
)

# FALSE videoları işle
false_count = batch_process_videos(
    FALSE_VIDEOS_DIR, 
    f'{OUTPUT_DIR}/csv/false', 
    'false', 
    'YANLIŞ FORM (false)'
)

print(f"\n{'='*60}")
print(f"TOPLAM: {true_count + false_count} video işlendi")
print(f"{'='*60}")

## 📊 5. Feature Extraction (16 Özellik)

In [ ]:
def extract_features(csv_path: str) -> Dict[str, float]:
    """
    CSV'den 16 biyomekanik özellik çıkart:
    - rom: Range of Motion
    - true_torso_stability_left_mean/std: Sol gövde stabilitesi
    - true_torso_stability_right_mean/std: Sağ gövde stabilitesi
    - bilateral_true_torso_mean/std: İki taraflı gövde stabilitesi
    - tempo: Rep başına süre
    - symmetry: Sol-sağ simetri
    - movement_smoothness: Hareket akıcılığı
    - peak_flexion/extension: Min/max açılar
    - total_reps: Toplam rep
    - correct_rep_ratio: Doğru rep oranı
    - angle_cv: Açı varyasyon katsayısı
    - frame_count: Frame sayısı
    """
    try:
        df = pd.read_csv(csv_path)
        features = {}
        
        # 1. ROM
        left_angles = pd.to_numeric(df['left_angle_smoothed_deg'], errors='coerce').dropna()
        right_angles = pd.to_numeric(df['right_angle_smoothed_deg'], errors='coerce').dropna()
        all_angles = pd.concat([left_angles, right_angles])
        features['rom'] = float(all_angles.max() - all_angles.min()) if len(all_angles) > 0 else 0.0
        
        # 2-3. True Torso Stability - Left
        left_torso = pd.to_numeric(df['left_true_torso_angle_deg'], errors='coerce').dropna()
        features['true_torso_stability_left_mean'] = float(left_torso.mean()) if len(left_torso) > 0 else 0.0
        features['true_torso_stability_left_std'] = float(left_torso.std()) if len(left_torso) > 0 else 0.0
        
        # 4. Tempo
        total_reps = df['total_reps'].iloc[-1] if len(df) > 0 else 0
        total_time = df['time_s'].iloc[-1] if len(df) > 0 else 0
        features['tempo'] = float(total_time / total_reps) if total_reps > 0 else 0.0
        
        # 5. Symmetry
        if len(left_angles) > 1 and len(right_angles) > 1:
            min_len = min(len(left_angles), len(right_angles))
            corr = np.corrcoef(left_angles.iloc[:min_len].values, right_angles.iloc[:min_len].values)[0, 1]
            features['symmetry'] = float(corr) if not np.isnan(corr) else 0.0
        else:
            features['symmetry'] = 0.0
        
        # 6-7. True Torso Stability - Right
        right_torso = pd.to_numeric(df['right_true_torso_angle_deg'], errors='coerce').dropna()
        features['true_torso_stability_right_mean'] = float(right_torso.mean()) if len(right_torso) > 0 else 0.0
        features['true_torso_stability_right_std'] = float(right_torso.std()) if len(right_torso) > 0 else 0.0
        
        # 8-9. Bilateral True Torso
        all_torso = pd.concat([left_torso, right_torso])
        features['bilateral_true_torso_mean'] = float(all_torso.mean()) if len(all_torso) > 0 else 0.0
        features['bilateral_true_torso_std'] = float(all_torso.std()) if len(all_torso) > 0 else 0.0
        
        # 10. Movement Smoothness
        all_changes = pd.concat([left_angles.diff().abs().dropna(), right_angles.diff().abs().dropna()])
        avg_change = all_changes.mean()
        features['movement_smoothness'] = float(1.0 / (avg_change + 1e-6)) if len(all_changes) > 0 else 0.0
        
        # 11-12. Peak Flexion/Extension
        features['peak_flexion'] = float(all_angles.min()) if len(all_angles) > 0 else 0.0
        features['peak_extension'] = float(all_angles.max()) if len(all_angles) > 0 else 0.0
        
        # 13. Total Reps
        features['total_reps'] = float(total_reps)
        
        # 14. Correct Rep Ratio
        left_correct = df['left_correct_reps'].iloc[-1] if len(df) > 0 else 0
        right_correct = df['right_correct_reps'].iloc[-1] if len(df) > 0 else 0
        total_correct = left_correct + right_correct
        total_reps_both = total_reps * 2
        features['correct_rep_ratio'] = float(total_correct / total_reps_both) if total_reps_both > 0 else 0.0
        
        # 15. Angle CV
        angle_mean = all_angles.mean()
        angle_std = all_angles.std()
        features['angle_cv'] = float(angle_std / angle_mean) if angle_mean > 0 else 0.0
        
        # 16. Frame Count
        features['frame_count'] = float(len(df))
        
        return features
        
    except Exception as e:
        print(f"Hata: {e}")
        return {k: 0.0 for k in ['rom', 'true_torso_stability_left_mean', 'true_torso_stability_left_std',
                                  'tempo', 'symmetry', 'true_torso_stability_right_mean', 
                                  'true_torso_stability_right_std', 'bilateral_true_torso_mean',
                                  'bilateral_true_torso_std', 'movement_smoothness', 'peak_flexion',
                                  'peak_extension', 'total_reps', 'correct_rep_ratio', 'angle_cv', 'frame_count']}

print("✓ Feature extraction fonksiyonu yüklendi!")

## 📈 6. Veri Yükleme ve Feature Çıkarımı

In [ ]:
def load_data(csv_base_dir):
    """Tüm CSV'lerden feature çıkart ve veri seti oluştur"""
    
    true_csvs = sorted(glob.glob(os.path.join(csv_base_dir, 'true', 'output_true_*.csv')))
    false_csvs = sorted(glob.glob(os.path.join(csv_base_dir, 'false', 'output_false_*.csv')))
    
    print(f"Bulunan CSV'ler:")
    print(f"  TRUE: {len(true_csvs)}")
    print(f"  FALSE: {len(false_csvs)}")
    
    features_list = []
    labels = []
    filenames = []
    
    # TRUE samples (label = 1)
    print("\nTRUE örnekler işleniyor...")
    for csv_path in true_csvs:
        features = extract_features(csv_path)
        features_list.append(features)
        labels.append(1)
        filenames.append(os.path.basename(csv_path))
    
    # FALSE samples (label = 0)
    print("FALSE örnekler işleniyor...")
    for csv_path in false_csvs:
        features = extract_features(csv_path)
        features_list.append(features)
        labels.append(0)
        filenames.append(os.path.basename(csv_path))
    
    df_features = pd.DataFrame(features_list)
    X = df_features.values
    y = np.array(labels)
    
    print(f"\nVeri seti boyutu: {X.shape}")
    print(f"Özellikler: {list(df_features.columns)}")
    
    return X, y, filenames, df_features.columns.tolist(), df_features

# Veriyi yükle
X, y, filenames, feature_names, df_features = load_data(f'{OUTPUT_DIR}/csv')

print("\n✓ Veri yüklendi!")
print(f"  Toplam örnek: {len(y)}")
print(f"  TRUE (doğru form): {sum(y)}")
print(f"  FALSE (yanlış form): {len(y) - sum(y)}")

In [ ]:
# Feature istatistikleri
print("\n📊 FEATURE İSTATİSTİKLERİ:")
print("="*60)
df_features.describe().round(2)

## 🤖 7. RandomForest Model Eğitimi

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import joblib

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Eğitim seti: {len(X_train)} örnek")
print(f"Test seti: {len(X_test)} örnek")
print(f"Eğitim dağılımı: TRUE={sum(y_train)}, FALSE={len(y_train)-sum(y_train)}")
print(f"Test dağılımı: TRUE={sum(y_test)}, FALSE={len(y_test)-sum(y_test)}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# RandomForest eğitimi
print("\n🤖 RandomForest eğitiliyor...")
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight='balanced'
)

model.fit(X_train_scaled, y_train)
print("✓ Model eğitildi!")

## 📊 8. Model Değerlendirmesi

In [ ]:
# Tahminler
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n{'='*60}")
print(f"MODEL DEĞERLENDİRMESİ")
print(f"{'='*60}")
print(f"\n🎯 Accuracy: {accuracy:.2%}")

# Classification Report
print(f"\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Yanlış Form (0)', 'Doğru Form (1)']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\n📊 Confusion Matrix:")
print(f"                  Tahmin")
print(f"                  0        1")
print(f"Gerçek  0        {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"        1        {cm[1,0]:3d}      {cm[1,1]:3d}")

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📈 FEATURE IMPORTANCE (Önem Sırasına Göre):")
print("="*60)
print(feature_importance.to_string(index=False))

# Görselleştirme
plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance)), feature_importance['importance'].values, color='steelblue')
plt.yticks(range(len(feature_importance)), feature_importance['feature'].values)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance - Biceps Curl Form Classification', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Grafik kaydedildi: {OUTPUT_DIR}/feature_importance.png")

## 💾 9. Modeli Kaydet

In [ ]:
# Model ve scaler'ı kaydet
model_path = f'{OUTPUT_DIR}/biceps_model.pkl'
scaler_path = f'{OUTPUT_DIR}/scaler.pkl'

joblib.dump(model, model_path)
joblib.dump(scaler, scaler_path)

# Feature importance CSV
feature_importance.to_csv(f'{OUTPUT_DIR}/feature_importance.csv', index=False)

print(f"\n{'='*60}")
print(f"✓ MODEL KAYDEDİLDİ")
print(f"{'='*60}")
print(f"  Model: {model_path}")
print(f"  Scaler: {scaler_path}")
print(f"  Feature Importance: {OUTPUT_DIR}/feature_importance.csv")

## 📥 10. Dosyaları İndir

In [ ]:
from google.colab import files

# Model dosyalarını indir
print("Model dosyaları indiriliyor...")
files.download(model_path)
files.download(scaler_path)
files.download(f'{OUTPUT_DIR}/feature_importance.csv')
files.download(f'{OUTPUT_DIR}/feature_importance.png')

print("\n✓ Tüm dosyalar indirildi!")

## 🔮 11. Yeni Video Test Etme (Opsiyonel)

In [ ]:
def predict_video(video_path):
    """Yeni bir video için form skoru hesapla"""
    
    print(f"Video işleniyor: {video_path}")
    
    # Geçici CSV oluştur
    temp_csv = '/content/temp_video.csv'
    extractor = VideoAngleExtractor()
    df = extractor.process_video(video_path, temp_csv)
    
    # Feature çıkart
    features = extract_features(temp_csv)
    
    # Tahmin yap
    feature_order = ['rom', 'true_torso_stability_left_mean', 'true_torso_stability_left_std',
                     'tempo', 'symmetry', 'true_torso_stability_right_mean', 
                     'true_torso_stability_right_std', 'bilateral_true_torso_mean',
                     'bilateral_true_torso_std', 'movement_smoothness', 'peak_flexion',
                     'peak_extension', 'total_reps', 'correct_rep_ratio', 'angle_cv', 'frame_count']
    
    X_new = np.array([[features[k] for k in feature_order]])
    X_new_scaled = scaler.transform(X_new)
    
    proba = model.predict_proba(X_new_scaled)[0]
    form_score = proba[1] * 100
    
    # Grade
    if form_score >= 80:
        grade = "Perfect 🌟"
    elif form_score >= 60:
        grade = "Good 👍"
    elif form_score >= 40:
        grade = "Fair ⚠️"
    else:
        grade = "Poor ❌"
    
    print(f"\n{'='*50}")
    print(f"📊 FORM DEĞERLENDİRMESİ")
    print(f"{'='*50}")
    print(f"  Form Skoru: {form_score:.1f}%")
    print(f"  Grade: {grade}")
    print(f"  ROM: {features['rom']:.1f}°")
    print(f"  Total Reps: {int(features['total_reps'])}")
    print(f"{'='*50}")
    
    return form_score, grade, features

# Kullanım:
# predict_video('/content/drive/MyDrive/test_video.mp4')

---
## 📋 Özet

Bu notebook ile:
1. ✅ Videolardan MediaPipe ile açılar çıkarıldı
2. ✅ 16 biyomekanik özellik hesaplandı
3. ✅ RandomForest Classifier eğitildi
4. ✅ Model değerlendirildi ve kaydedildi

### Çıktı Dosyaları:
- `biceps_model.pkl` - Eğitilmiş model
- `scaler.pkl` - Feature scaler
- `feature_importance.csv` - Feature önem sıralaması
- `feature_importance.png` - Görselleştirme